###Dataset info
This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers. \

Dataset contains 1067371 rows and 8 columns.

###Overview of data
- InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.
- UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).
- CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal. The name of the country where a customer resides.

In [3]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=cf44c7216df7d16df3718993106030f1c9fb47e3866ec580ae81e727ca6b8331
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


##EDA
1. Первым этапом загружаем датасет.  \

- **data_1** -  данные за 2009 и 2010 года
- **data_2** - данные за 2010 и 2011 года





In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import wget

url = 'https://archive.ics.uci.edu/static/public/502/online+retail+ii.zip'
filename = wget.download(url) #returns file name

with zipfile.ZipFile(filename, 'r') as zip_ref: #read mode
    zip_ref.extractall('./')

Data_1 = pd.read_excel('online_retail_II.xlsx', sheet_name='Year 2009-2010') #use data from list 1(Year 2009-2010)
Data_2 = pd.read_excel('online_retail_II.xlsx', sheet_name='Year 2010-2011') #use data from list 2(Year 2010-2011)

In [5]:
data_1 = Data_1
data_2 = Data_2
data_size = len(data_1)+len(data_2)

2. Обрабатываем значения в датасете.
    - находим пропущенные значения

In [9]:
null_1 = data_1.isnull().sum() #series: the first coloumn - names of coloumns from data_1,
                               #the second coloumn - the quantity of empty cells for every attribute
perc = null_1 / len(data_1)
null_data_1 = pd.concat([null_1, perc], axis=1, keys=['num_null_values', 'percentage_null_vaLues'])
display(null_data_1)

null_2 = data_2.isnull().sum()
perc = null_2 / len(data_2)
null_data_2 = pd.concat([null_2, perc], axis=1, keys=['num_null_values', 'percentage_null_vaLues'])
display(null_data_2)


print((null_data_1.iat[6, 0] + null_data_2.iat[6,0])/(len(data_1)+len(data_2))*100)

,num_null_values,percentage_null_vaLues
Invoice,0,0.000000
StockCode,0,0.000000
Description,2928,0.005572
Quantity,0,0.000000
InvoiceDate,0,0.000000
Price,0,0.000000
Customer ID,107927,0.205395
Country,0,0.000000


,num_null_values,percentage_null_vaLues
Invoice,0,0.000000
StockCode,0,0.000000
Description,1454,0.002683
Quantity,0,0.000000
InvoiceDate,0,0.000000
Price,0,0.000000
Customer ID,135080,0.249266
Country,0,0.000000


22.766872999172733


Выяснили, что отсутствует около 20-24% данный Сustomer ID в обои датафреймах, что составляет примерно 22,7% от всей бд. Для задачи классификации  RFM нам вполне будет достаточно оставшихся данных, так что при построении этой модели удалим строки с пропушенными значениями Сustomer ID.

   - Выявляем выбросы и аномалии.

   Из аномалий датасете есть отмененные транзакции.  Найдем их количество.

In [ ]:
data1 = data_1
data1['t'] = data_1.Invoice.apply(lambda x: type(x))
data1['t'] = data1['t'] == int
print(len(data1.loc[data1.t == False])/data_size)

data2 = data_2
data2['t'] = data_2.Invoice.apply(lambda x: type(x))
data2['t'] = data2['t'] == int
print(len(data2.loc[data2.t == False])/data_size)

0.009564621860627655
0.00870456476707724


  Отмененных транзакций меньше двух процентов, поэтому удалим их, так как не известна причина отметы, из-за чего они могут помешать построению точной модели.

In [ ]:
data_1['t'] = data1['t']
data_1 = data_1.loc[data_1.t == True]
data_1 = data_1.drop(columns=['t'])
print(f'Number of rows in data_1: {len(data_1)}')

data_2['t'] = data2['t']
data_2 = data_2.loc[data_2.t == True]
data_2 = data_2.drop(columns=['t'])
print(f'Number of rows in data_2: {len(data_2)}')

data_size = len(data_1) + len(data_2)
print(f'Total datasset size: {data_size}')

Number of rows in data_1: 515252
Number of rows in data_2: 532619
Total datasset size: 1047871


   - Поищем аномалии в столбцах Quantity и Price

In [ ]:
#отрицательное кол-во товара
print(f"data_1 percentage of not positive quantity: {len(data_1.loc[data_1.Quantity <= 0])/data_size}")
print(f'data_2 percentage of not positive quantity: {len(data_2.loc[data_2.Quantity <= 0])/data_size}', '\n')

#отрицательная цена товара
print(f'data_1 percentage of negative prices: {len(data_1.loc[data_1.Price < 0])/data_size}')
print(f'data_2 percentage of negative prices: {len(data_2.loc[data_2.Price < 0])/data_size}')

data_1 percentage of not positive quantity: 0.0020241041120519607
data_2 percentage of not positive quantity: 0.001274966097926176 

data_1 percentage of negative prices: 0.0
data_2 percentage of negative prices: 0.0


 Удалим строки с отрицательными Quantity, так как их количеество меньше одного рпоцента.

In [ ]:
data_1 = data_1.loc[data_1.Quantity >= 0]
print(f'Number of rows in data_1: {len(data_1)}')

data_2 = data_2.loc[data_2.Quantity >= 0]
print(f'Number of rows in data_2: {len(data_2)}')

data_size = len(data_1) + len(data_2)
print(f'Total datasset size: {data_size}')

Number of rows in data_1: 513131
Number of rows in data_2: 531283
Total datasset size: 1044414


 -  Удалим дубликаты

In [ ]:
data_1 = data_1.drop_duplicates()
print(f'Number of rows in data_1: {len(data_1)}')

data_2 = data_2.drop_duplicates()
print(f'Number of rows in data_2: {len(data_2)}')

data_size = len(data_1) + len(data_2)
print(f'Total datasset size: {data_size}')

Number of rows in data_1: 506290
Number of rows in data_2: 526052
Total datasset size: 1032342


  - Найдем выбросы и удалим их. Чтобы понять, является ли значение выбросом, воспользуемся интерквартильным размахом:
  $$
IRL=Q_3-Q_1
  $$
  где $Q_1$ — первая квартиль — такое значение признака, меньше которого ровно 25% всех значений признаков. $Q_3$ — третья квартиль — значение, меньше которого ровно 75% всех значений признака. \
  Выбросы будут лежать вне данного интервала $[ Q_1 -1.5IQR, Q_3 + 1.5IQR ]$.

In [ ]:
def IRL(data, var):
  q_1 = data[var].quantile(.25)
  q_3 = data[var].quantile(.75)
  irl = q_3-q_1
  bottom = q_1 - 1.5*irl
  top = q_3 + 1.5*irl
  data = data.loc[data[var] <= top]
  data = data.loc[data[var] >= bottom]
  return data

data_1 = IRL(data_1, 'Quantity')
data_1 = IRL(data_1, 'Price')
data_2 = IRL(data_1, 'Quantity')
data_2 = IRL(data_1, 'Price')
data_size = len(data_1) + len(data_2)

print(f'Number of rows in data_1: {len(data_1)}')
print(f'Number of rows in data_1: {len(data_2)}')
print(f'Total datasset size: {data_size}')

Number of rows in data_1: 446396
Number of rows in data_1: 424500
Total datasset size: 870896


**Данные готовы**
